# LG-CoTrain: Alternative Early Stopping — Full Experiment Run

This notebook runs the **complete experiment sweep** (all budgets × all seed sets × all events)
under each of the 6 stopping strategies, then produces the same comparison charts as
notebook 04 but with statistically reliable mean ± std estimates.

| Strategy | Key Idea |
|---|---|
| `baseline` | Original: stop when ensemble macro-F1 plateaus for `patience` epochs |
| `no_early_stopping` | Run all `finetune_max_epochs`; restore best-ever checkpoint (upper bound) |
| `per_class_patience` | Stop only when **every** class F1 has individually plateaued |
| `weighted_macro_f1` | Weight rare classes more in the stopping metric |
| `balanced_dev` | Resample dev set to equal class sizes for the stopping signal |
| `scaled_threshold` | Require a larger improvement delta for highly imbalanced events |

**Total experiments**: 6 strategies × 10 events × 4 budgets × 3 seeds = **720 runs**

Results are stored in `results/stop-{strategy}/`.
See **notebook 04** for a fast preview using budget=50, seed=1 only.

## Resume Support

Every `(event, budget, seed_set)` combination is written to its own `metrics.json`
as soon as it completes. Re-running any cell automatically skips completed experiments.

In [ ]:
import json
import statistics
import sys
import time
from pathlib import Path


def _find_repo_root(marker: str = "lg_cotrain") -> Path:
    for candidate in [Path().resolve()] + list(Path().resolve().parents):
        if (candidate / marker).is_dir():
            return candidate
    raise RuntimeError(
        f"Cannot find repo root: no ancestor directory contains '{marker}/'. "
        "Run the notebook from inside the repository."
    )


repo_root = _find_repo_root()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import matplotlib.pyplot as plt
import numpy as np

from lg_cotrain.run_all import BUDGETS, SEED_SETS, format_summary_table, run_all_experiments


class ProgressTracker:
    """Track global progress across all strategies × events × budgets × seeds."""

    def __init__(self, total: int, already_done: int, start_time: float):
        self.total = total
        self.done = already_done
        self.start_time = start_time

    def update(self, event, budget, seed_set, status):
        self.done += 1
        elapsed = time.time() - self.start_time
        pct = 100.0 * self.done / self.total if self.total else 0
        elapsed_h = elapsed / 3600
        remaining = self.total - self.done
        eta_h = (elapsed / self.done) * remaining / 3600 if self.done > 0 else 0
        print(
            f"  [PROGRESS] {self.done}/{self.total} ({pct:.1f}%)"
            f"  |  Elapsed: {elapsed_h:.2f}h  |  ETA: {eta_h:.2f}h  |  {status}"
        )


print(f"Repo root: {repo_root}")
print(f"Budgets  : {BUDGETS}")
print(f"Seed sets: {SEED_SETS}")

In [ ]:
# ---- Configuration ----

PSEUDO_LABEL_SOURCE = "gpt-4o"

STRATEGIES = [
    "baseline",
    "no_early_stopping",
    "per_class_patience",
    "weighted_macro_f1",
    "balanced_dev",
    "scaled_threshold",
]

DATA_ROOT = str(repo_root / "data")

# Discover all events
TARGET_EVENTS = sorted(
    p.name for p in (Path(DATA_ROOT) / "original").iterdir() if p.is_dir()
)

# Each strategy gets its own results sub-folder
STRATEGY_RESULTS_ROOTS = {
    s: str(repo_root / "results" / f"stop-{s}")
    for s in STRATEGIES
}

expts_per_strategy = len(TARGET_EVENTS) * len(BUDGETS) * len(SEED_SETS)
total_runs = len(STRATEGIES) * expts_per_strategy

print(f"Strategies : {STRATEGIES}")
print(f"Events     : {len(TARGET_EVENTS)} events")
print(f"Per strategy: {expts_per_strategy} experiments  ({len(TARGET_EVENTS)} events × {len(BUDGETS)} budgets × {len(SEED_SETS)} seeds)")
print(f"Grand total : {total_runs} experiments")
print()
for s, r in STRATEGY_RESULTS_ROOTS.items():
    print(f"  {s:<25} → {r}")

## Running Experiments

The loop below processes one strategy at a time, running all events × budgets × seeds
for each. Completed experiments are skipped automatically.

> **Runtime estimate**: ~3-5 hours per strategy on GPU, ~18-30 hours total.
> Run overnight or split across sessions — resume support ensures no work is lost.

In [ ]:
# Count already-completed experiments across all strategies (for accurate ETA from the start)
already_done = sum(
    1
    for strategy in STRATEGIES
    for event in TARGET_EVENTS
    for budget in BUDGETS
    for seed_set in SEED_SETS
    if (
        Path(STRATEGY_RESULTS_ROOTS[strategy])
        / event / f"{budget}_set{seed_set}" / "metrics.json"
    ).exists()
)
total_experiments = len(STRATEGIES) * expts_per_strategy

print(f"Total experiments : {total_experiments}")
print(f"Already completed : {already_done}")
print(f"Remaining         : {total_experiments - already_done}")
print()

overall_start = time.time()
tracker = ProgressTracker(total_experiments, already_done, overall_start)
all_strategy_results = {}  # strategy -> event -> list[result_dict]

for strategy in STRATEGIES:
    results_root = STRATEGY_RESULTS_ROOTS[strategy]
    strat_start = time.time()
    print(f"\n{'=' * 70}")
    print(f"Strategy: {strategy}  →  {results_root}")
    print(f"{'=' * 70}")
    all_strategy_results[strategy] = {}

    for event in TARGET_EVENTS:
        print(f"\n  Event: {event}")
        results = run_all_experiments(
            event,
            pseudo_label_source=PSEUDO_LABEL_SOURCE,
            stopping_strategy=strategy,
            data_root=DATA_ROOT,
            results_root=results_root,
            _on_experiment_done=tracker.update,
        )
        all_strategy_results[strategy][event] = results
        print()
        print(format_summary_table(results, event))

    strat_elapsed = time.time() - strat_start
    print(f"\n  Strategy '{strategy}' done in {strat_elapsed / 3600:.2f}h ({strat_elapsed / 60:.1f}min)")

total_elapsed = time.time() - overall_start
print(f"\n{'=' * 70}")
print(f"All strategies complete in {total_elapsed / 3600:.2f}h ({total_elapsed / 60:.1f}min)")

In [ ]:
# Load any results that already existed (re-run safe)
for strategy in STRATEGIES:
    results_root = Path(STRATEGY_RESULTS_ROOTS[strategy])
    if strategy not in all_strategy_results:
        all_strategy_results[strategy] = {}

    for event in TARGET_EVENTS:
        if event in all_strategy_results[strategy]:
            continue
        results = []
        for budget in BUDGETS:
            for seed_set in SEED_SETS:
                path = results_root / event / f"{budget}_set{seed_set}" / "metrics.json"
                if path.exists():
                    with open(path) as f:
                        results.append(json.load(f))
        if results:
            all_strategy_results[strategy][event] = results

# Build lookup: lookup[strategy][event][(budget, seed_set)] -> result
lookup = {}
for strategy in STRATEGIES:
    lookup[strategy] = {}
    for event in TARGET_EVENTS:
        lookup[strategy][event] = {}
        for r in all_strategy_results.get(strategy, {}).get(event, []):
            if r is not None:
                lookup[strategy][event][(r["budget"], r["seed_set"])] = r

# Coverage report
print("Experiments loaded per strategy:")
for strategy in STRATEGIES:
    n = sum(len(lookup[strategy][e]) for e in TARGET_EVENTS)
    expected = expts_per_strategy
    pct = 100 * n / expected if expected else 0
    print(f"  {strategy:<25}: {n:>3}/{expected}  ({pct:.0f}%)")

In [ ]:
# Summary tables: for each event, show mean macro-F1 per strategy × budget
# plus a delta-from-baseline table.

col_w = 10

for event in TARGET_EVENTS:
    print(f"\n{'=' * 70}")
    print(f"Event: {event}")
    print(f"{'=' * 70}")

    # --- Absolute macro-F1 ---
    print(f"{'Strategy':<26}" + "".join(f" B={b:<{col_w - 2}}" for b in BUDGETS) + " | Mean")
    print("-" * (26 + col_w * len(BUDGETS) + 7))

    baseline_means = {}
    for strategy in STRATEGIES:
        row = f"{strategy:<26}"
        budget_means = []
        for budget in BUDGETS:
            f1s = [
                lookup[strategy][event].get((budget, s), {}).get("test_macro_f1")
                for s in SEED_SETS
            ]
            f1s = [f for f in f1s if f is not None]
            if f1s:
                m = statistics.mean(f1s)
                sd = statistics.stdev(f1s) if len(f1s) >= 2 else 0.0
                budget_means.append(m)
                row += f" {m:.4f}±{sd:.4f}"
            else:
                budget_means.append(None)
                row += f" {'N/A':<{col_w}}"
        valid = [v for v in budget_means if v is not None]
        row += f" | {statistics.mean(valid):.4f}" if valid else " | N/A"
        print(row)
        if strategy == "baseline":
            baseline_means = dict(zip(BUDGETS, budget_means))

    # --- Delta vs baseline ---
    print()
    print(f"Delta vs baseline  (+) = better:")
    print(f"{'Strategy':<26}" + "".join(f" B={b:<{col_w - 2}}" for b in BUDGETS) + " | Mean Δ")
    print("-" * (26 + col_w * len(BUDGETS) + 9))

    for strategy in STRATEGIES:
        if strategy == "baseline":
            continue
        row = f"{strategy:<26}"
        deltas = []
        for budget in BUDGETS:
            f1s = [
                lookup[strategy][event].get((budget, s), {}).get("test_macro_f1")
                for s in SEED_SETS
            ]
            f1s = [f for f in f1s if f is not None]
            base = baseline_means.get(budget)
            if f1s and base is not None:
                d = statistics.mean(f1s) - base
                deltas.append(d)
                sign = "+" if d >= 0 else ""
                row += f" {sign}{d:.4f}   "
            else:
                row += f" {'N/A':<{col_w}}"
        row += f" | {'+' if sum(deltas)/len(deltas)>=0 else ''}{sum(deltas)/len(deltas):.4f}" if deltas else " | N/A"
        print(row)

In [ ]:
# Grouped bar chart: macro-F1 by budget, grouped bars per strategy
# One subplot per event (mean ± std across 3 seeds)

n_events     = len(TARGET_EVENTS)
n_strategies = len(STRATEGIES)
bar_width    = 0.8 / n_strategies
colors       = plt.cm.tab10(np.linspace(0, 1, n_strategies))

# Layout: up to 5 events per row
ncols = min(5, n_events)
nrows = (n_events + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(4.5 * ncols, 4.5 * nrows), sharey=False)
axes_flat = np.array(axes).flatten() if n_events > 1 else [axes]

for ax, event in zip(axes_flat, TARGET_EVENTS):
    x = np.arange(len(BUDGETS))
    for i, (strategy, color) in enumerate(zip(STRATEGIES, colors)):
        means, errs = [], []
        for budget in BUDGETS:
            f1s = [
                lookup[strategy][event].get((budget, s), {}).get("test_macro_f1")
                for s in SEED_SETS
            ]
            f1s = [f for f in f1s if f is not None]
            means.append(statistics.mean(f1s) if f1s else 0)
            errs.append(statistics.stdev(f1s) if len(f1s) >= 2 else 0)
        offset = (i - n_strategies / 2 + 0.5) * bar_width
        ax.bar(
            x + offset, means, bar_width * 0.9,
            yerr=errs, capsize=2,
            label=strategy, color=color, alpha=0.85,
        )
    ax.set_title(event.replace("_", " ").title(), fontsize=9)
    ax.set_xlabel("Budget")
    ax.set_ylabel("Macro-F1")
    ax.set_xticks(x)
    ax.set_xticklabels([str(b) for b in BUDGETS])
    ax.set_ylim(0, 1)
    ax.grid(axis="y", alpha=0.3)

# Hide unused subplots
for ax in axes_flat[n_events:]:
    ax.set_visible(False)

axes_flat[0].legend(fontsize=7, loc="upper left", framealpha=0.7)
fig.suptitle(
    f"Stopping Strategy Comparison — All Budgets & Seeds\n(pseudo-labels: {PSEUDO_LABEL_SOURCE})",
    fontsize=12,
)
plt.tight_layout()
plt.show()

In [ ]:
# Cross-event summary: for each strategy, show mean macro-F1 across all events and budgets.
# Useful for picking a single "best" strategy to recommend.

print("Grand summary — mean macro-F1 across all events and seeds\n")
print(f"{'Strategy':<26}" + "".join(f" B={b:<8}" for b in BUDGETS) + " | Overall")
print("-" * (26 + 10 * len(BUDGETS) + 10))

for strategy in STRATEGIES:
    row = f"{strategy:<26}"
    all_f1s = []
    for budget in BUDGETS:
        f1s = [
            lookup[strategy][e].get((budget, s), {}).get("test_macro_f1")
            for e in TARGET_EVENTS
            for s in SEED_SETS
        ]
        f1s = [f for f in f1s if f is not None]
        if f1s:
            m = statistics.mean(f1s)
            sd = statistics.stdev(f1s) if len(f1s) >= 2 else 0
            all_f1s.extend(f1s)
            row += f" {m:.4f}±{sd:.4f}"
        else:
            row += f" {'N/A':<10}"
    overall = statistics.mean(all_f1s) if all_f1s else None
    row += f" | {overall:.4f}" if overall is not None else " | N/A"
    print(row)

# Per-class F1 heatmap for each event at budget=5 (hardest case)
from lg_cotrain.data_loading import CLASS_LABELS

HEATMAP_BUDGET = 5
print(f"\nPer-class F1 heatmaps at budget={HEATMAP_BUDGET} (hardest imbalance scenario)")

for event in TARGET_EVENTS:
    strategies_with_data = []
    class_f1_matrix = []

    for strategy in STRATEGIES:
        per_class_all_seeds = [
            lookup[strategy][event][(HEATMAP_BUDGET, s)]["test_per_class_f1"]
            for s in SEED_SETS
            if (HEATMAP_BUDGET, s) in lookup[strategy][event]
            and "test_per_class_f1" in lookup[strategy][event][(HEATMAP_BUDGET, s)]
        ]
        if per_class_all_seeds:
            mean_per_class = [
                statistics.mean(seed[i] for seed in per_class_all_seeds)
                for i in range(len(per_class_all_seeds[0]))
            ]
            strategies_with_data.append(strategy)
            class_f1_matrix.append(mean_per_class)

    if not strategies_with_data:
        print(f"  No per-class data for {event} at budget={HEATMAP_BUDGET}, skipping.")
        continue

    data = np.array(class_f1_matrix)
    fig, ax = plt.subplots(
        figsize=(max(9, len(CLASS_LABELS) * 0.75), len(strategies_with_data) * 0.65 + 1.8)
    )
    im = ax.imshow(data, cmap="RdYlGn", aspect="auto", vmin=0, vmax=1)

    ax.set_xticks(range(len(CLASS_LABELS)))
    ax.set_xticklabels(CLASS_LABELS, rotation=45, ha="right", fontsize=8)
    ax.set_yticks(range(len(strategies_with_data)))
    ax.set_yticklabels(strategies_with_data, fontsize=9)
    ax.set_title(
        f"{event}  |  Budget={HEATMAP_BUDGET}  |  Per-class F1 (mean across seeds)",
        fontsize=10,
    )

    for i in range(len(strategies_with_data)):
        for j in range(len(CLASS_LABELS)):
            val = data[i, j]
            color = "black" if 0.25 < val < 0.75 else "white"
            ax.text(j, i, f"{val:.2f}", ha="center", va="center", fontsize=7, color=color)

    fig.colorbar(im, ax=ax, label="F1 Score")
    plt.tight_layout()
    plt.show()

In [ ]:
# Rebuild multi-tab dashboard with all strategy result sets.

from lg_cotrain.dashboard import discover_result_sets, generate_html_multi

TOP_RESULTS_ROOT = str(repo_root / "results")

result_sets = discover_result_sets(TOP_RESULTS_ROOT)
print(f"Discovered {len(result_sets)} result set(s):")
for name, path in result_sets.items():
    print(f"  {name:<35} → {path}")

html = generate_html_multi(result_sets, data_root=DATA_ROOT)
dashboard_path = Path(TOP_RESULTS_ROOT) / "dashboard.html"
dashboard_path.write_text(html)
print(f"\nDashboard written to: {dashboard_path}")
print("Open in a browser to compare all strategies across all budgets and events.")